In [24]:
from pprint import pprint
import requests
import pandas as pd
import numpy as np
from time import sleep
from tqdm import tqdm
import warnings
from urllib3.exceptions import InsecureRequestWarning # 针对ssl验证产生警告的错书

In [25]:
warnings.filterwarnings("ignore", category=InsecureRequestWarning)

In [26]:
base_url = 'https://data.stats.gov.cn/adv.htm'
param = {'m':'wd', 'db':'fsnd'}
r = requests.get(base_url, params=param, verify=False)
pprint(r.json())

[{'code': 'zb', 'fsort': 1, 'issj': False, 'name': '指标'},
 {'code': 'reg', 'fsort': 2, 'issj': False, 'name': '地区'},
 {'code': 'sj', 'fsort': 3, 'issj': True, 'name': '时间'}]


### 一级指标

In [27]:
base_url = 'https://data.stats.gov.cn/adv.htm'
param = {'m':'findZbXl', 'wd':'zb', 'db':'fsnd'}
r = requests.get(base_url, params=param, verify=False)
level1 = pd.DataFrame(r.json())

In [28]:
level1.to_csv('./data/level1.csv', index=False, encoding='utf-8')
level1.head(2)

,dbcode,exp,id,isParent,name,open,pid,wd
0,fsnd,,A01,True,综合,False,,zb
1,fsnd,,A02,True,国民经济核算,False,,zb


### 二级指标

In [29]:
base_url = 'https://data.stats.gov.cn/adv.htm'
params = {'m':'findZbXl', 'wd':'zb', 'db':'fsnd'}
list_data = []
for each_dict in tqdm(level1.to_dict('records')):
    params['treeId'] = each_dict.get('id')
    r = requests.get(base_url, params=params, verify=False).json()
    list_data.append(pd.DataFrame(r))
    sleep(np.random.rand() * 0.1)
level2 = pd.concat(list_data, ignore_index=True)

100%|██████████| 28/28 [00:13<00:00,  2.02it/s]


In [30]:
level2.to_csv('./data/level2.csv',index=False, encoding='utf-8')
level2.head(2)

,dbcode,exp,id,isParent,name,open,pid,wd
0,fsnd,,A0101,False,行政区划,False,A01,zb
1,fsnd,,A0102,True,法人单位数,False,A01,zb
